In [12]:
from goodreads import client
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz, process
%matplotlib inline

### Importing File with tag counts for each tag for each book

In [3]:
book_tags = pd.read_csv('book_tags.csv')

In [4]:
book_tags.head()

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


In [5]:
len(book_tags.goodreads_book_id.unique())

10000

### Reading in the Tag Labels

In [7]:
tags = pd.read_csv('tags.csv')

In [9]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


### Importing list of 40 genres found on GoodReads website to match the tags with

In [10]:
genres = pd.read_csv('genres.csv', header=None)

In [11]:
genres.head()

,0
0,Art
1,Biography
2,Business
3,Chick Lit
4,Children's


### Fuzzy Matching of genre list to tags

In [13]:
genres = list(genres[0])

In [14]:
final_genres = {}
for i in genres:
    print(i)
    genres_i = process.extractBests(i, list(tags.tag_name), score_cutoff=96, limit = 25)
    final_genres[i] = genres_i

Art
Biography
Business
Chick Lit
Children's
Christian
Classics
Comics
Contemporary
Cookbooks
Crime
Ebooks
Fantasy
Fiction
Gay and Lesbian
Graphic Novels
Historical Fiction
History
Horror
Humor and Comedy
Manga
Memoir
Music
Mystery
Nonfiction
Paranormal
Philosophy
Poetry
Psychology
Religion
Romance
Science
Science Fiction
Self Help
Suspense
Spirituality
Sports
Thriller
Travel
Young Adult


In [15]:
final_genres

{'Art': [('art', 100)],
 'Biography': [('biography', 100)],
 'Business': [('business', 100)],
 'Chick Lit': [('chick-lit', 100)],
 "Children's": [('children-s', 100)],
 'Christian': [('christian', 100), ('christian-', 100)],
 'Classics': [('classics', 100)],
 'Comics': [('comics', 100)],
 'Contemporary': [('contemporary', 100), ('contempor\xe2\x80\x8bary', 100)],
 'Cookbooks': [('cookbooks', 100), ('cookbooks-', 100)],
 'Crime': [('crime', 100),
  ('crime-\xd8\xac\xd8\xb1\xd9\x8a\xd9\x85\xd8\xa9', 100)],
 'Ebooks': [('ebooks', 100)],
 'Fantasy': [('fantasy', 100)],
 'Fiction': [('-fiction', 100), ('fiction', 100), ('fiction-', 100)],
 'Gay and Lesbian': [('gay-and-lesbian', 100)],
 'Graphic Novels': [('graphic-novels', 100),
  ('graphics-novels', 97),
  ('graphic-novel', 96),
  ('graphicnovels', 96)],
 'Historical Fiction': [('historical-fiction', 100),
  ('historial-fiction', 97),
  ('historical-ficton', 97),
  ('historicalfiction', 97)],
 'History': [('-history', 100), ('history', 10

In [16]:
#Making a list from the tags
list_of_tags = []
for i in final_genres:
    for j in final_genres[i]:
        list_of_tags.append(j[0])

In [20]:
tags_genres_df = tags[tags.tag_name.isin(list_of_tags)]

In [22]:
tags_genres_df.head()

,tag_id,tag_name
25,25,-fiction
31,31,-history
42,42,-psychology-
2938,2938,art
4605,4605,biography


### Creating a dictionary mapping genres to tag names

In [23]:
from collections import defaultdict
dict_of_tags = defaultdict(list)
for i in final_genres:
    for j in final_genres[i]:
        dict_of_tags[i].append(j[0])

In [26]:
#Applying the mapping to our books with tags data
def get_genre(tag):
    for i in dict_of_tags:
        for j in dict_of_tags[i]:
            if tag == j:
                return i

In [27]:
tags_genres_df['genre'] = tags_genres_df.apply(lambda row: get_genre(row.tag_name), axis=1)

/Users/adamcoviensky/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [29]:
tags_genres_df.head()

,tag_id,tag_name,genre
25,25,-fiction,Fiction
31,31,-history,History
42,42,-psychology-,Psychology
2938,2938,art,Art
4605,4605,biography,Biography


In [30]:
book_genres = book_tags.merge(tags_genres_df, how = 'inner', left_on = 'tag_id', right_on = 'tag_id')

In [31]:
len(book_genres.goodreads_book_id.unique())

10000

In [32]:
book_genres.head()

,goodreads_book_id,tag_id,count,tag_name,genre
0,1,11305,37174,fantasy,Fantasy
1,2,11305,3441,fantasy,Fantasy
2,3,11305,47478,fantasy,Fantasy
3,5,11305,39330,fantasy,Fantasy
4,6,11305,38378,fantasy,Fantasy


In [33]:
genre_counts = book_genres.groupby('genre').agg('count')

In [34]:
genre_counts.reset_index(inplace=True)
genre_counts.head()

,genre,goodreads_book_id,tag_id,count,tag_name
0,Art,436,436,436,436
1,Biography,1109,1109,1109,1109
2,Business,377,377,377,377
3,Chick Lit,1974,1974,1974,1974
4,Children's,1267,1267,1267,1267


In [36]:
#Getting total tag counts corresponding to each genre for each book
book_genres = book_genres.groupby(by = ['goodreads_book_id', 'genre']).agg({'count' : 'sum'})

In [37]:
book_genres.index.levels

FrozenList([[1, 2, 3, 5, 6, 8, 10, 11, 13, 21, 24, 25, 26, 27, 28, 29, 30, 33, 34, 36, 50, 67, 93, 98, 105, 106, 112, 117, 119, 122, 231, 249, 250, 264, 275, 291, 295, 304, 320, 343, 348, 350, 353, 355, 357, 359, 360, 365, 378, 386, 400, 415, 424, 428, 431, 432, 446, 447, 449, 456, 475, 597, 599, 621, 629, 647, 656, 662, 665, 667, 668, 676, 698, 703, 706, 760, 771, 816, 822, 823, 825, 827, 830, 840, 865, 870, 880, 890, 900, 902, 903, 930, 952, 960, 968, 976, 998, 1005, 1032, 1052, ...], [u'Art', u'Biography', u'Business', u'Chick Lit', u'Children's', u'Christian', u'Classics', u'Comics', u'Contemporary', u'Cookbooks', u'Crime', u'Ebooks', u'Fantasy', u'Fiction', u'Gay and Lesbian', u'Graphic Novels', u'Historical Fiction', u'History', u'Horror', u'Humor and Comedy', u'Manga', u'Memoir', u'Music', u'Mystery', u'Nonfiction', u'Paranormal', u'Philosophy', u'Poetry', u'Psychology', u'Religion', u'Romance', u'Science', u'Science Fiction', u'Self Help', u'Spirituality', u'Sports', u'Suspense

In [38]:
book_genres.head()

count
goodreads_book_id genre              
1                 Children's     1095
                  Classics        958
                  Contemporary    289
                  Fantasy       37174
                  Fiction        9954

In [40]:
book_genres.to_csv('book_ids_to_genres.csv')

In [41]:
wide_book_genres = book_genres.pivot_table(index='goodreads_book_id', columns='genre', values = 'count')

In [42]:
wide_book_genres.head()

genre,Art,Biography,Business,Chick Lit,Children's,Christian,Classics,Comics,Contemporary,Cookbooks,...,Romance,Science,Science Fiction,Self Help,Spirituality,Sports,Suspense,Thriller,Travel,Young Adult
goodreads_book_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,1095.0,NaN,958.0,NaN,289.0,NaN,...,334.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12716.0
2,NaN,NaN,NaN,NaN,1098.0,NaN,NaN,NaN,NaN,NaN,...,213.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,865.0
3,NaN,NaN,NaN,NaN,1771.0,NaN,1898.0,NaN,397.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14984.0
5,NaN,NaN,NaN,NaN,1338.0,NaN,1090.0,NaN,324.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12856.0
6,NaN,NaN,NaN,NaN,1227.0,NaN,1032.0,NaN,301.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12979.0


In [43]:
null_counts = pd.isnull(wide_book_genres).sum(axis=1)

In [44]:
count=0
for i in null_counts:
    if i == 40:
        count +=1
print(count)

0


In [45]:
wide_book_genres.shape

(10000, 40)

In [46]:
wide_book_genres.fillna(0, inplace = True)

In [48]:
#Saving the final tag count version of 10k books with 40 genres and their counts
wide_book_genres.to_csv('books_with_tag_counts.csv')

### Mapping book genres to ratings book ids

In [50]:
wide_book_genres = pd.read_csv('books_with_tag_counts.csv')

In [51]:
wide_book_genres.head()

,goodreads_book_id,Art,Biography,Business,Chick Lit,Children's,Christian,Classics,Comics,Contemporary,...,Romance,Science,Science Fiction,Self Help,Spirituality,Sports,Suspense,Thriller,Travel,Young Adult
0,1,0.0,0.0,0.0,0.0,1095.0,0.0,958.0,0.0,289.0,...,334.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12716.0
1,2,0.0,0.0,0.0,0.0,1098.0,0.0,0.0,0.0,0.0,...,213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,865.0
2,3,0.0,0.0,0.0,0.0,1771.0,0.0,1898.0,0.0,397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14984.0
3,5,0.0,0.0,0.0,0.0,1338.0,0.0,1090.0,0.0,324.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12856.0
4,6,0.0,0.0,0.0,0.0,1227.0,0.0,1032.0,0.0,301.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12979.0


In [52]:
books = pd.read_csv('books.csv')

In [53]:
books.columns

Index([u'book_id', u'goodreads_book_id', u'best_book_id', u'work_id',
       u'books_count', u'isbn', u'isbn13', u'authors',
       u'original_publication_year', u'original_title', u'title',
       u'language_code', u'average_rating', u'ratings_count',
       u'work_ratings_count', u'work_text_reviews_count', u'ratings_1',
       u'ratings_2', u'ratings_3', u'ratings_4', u'ratings_5', u'image_url',
       u'small_image_url'],
      dtype='object')

In [54]:
good_cols = ['book_id', 'goodreads_book_id', 'authors', 'original_publication_year', 'language_code']

In [55]:
books = books.loc[:, good_cols]

In [56]:
books.head()

,book_id,goodreads_book_id,authors,original_publication_year,language_code
0,1,2767052,Suzanne Collins,2008.0,eng
1,2,3,"J.K. Rowling, Mary GrandPré",1997.0,eng
2,3,41865,Stephenie Meyer,2005.0,en-US
3,4,2657,Harper Lee,1960.0,eng
4,5,4671,F. Scott Fitzgerald,1925.0,eng


In [57]:
books_meta_info = books.merge(wide_book_genres, how='left', left_on='goodreads_book_id', right_on='goodreads_book_id')

In [58]:
books_meta_info.to_csv('books_meta_info.csv')

In [60]:
ratings = pd.read_csv('../ratings.csv')

In [61]:
ratings_meta_info = ratings.merge(books_meta_info, how='left', left_on='book_id', right_on='book_id')

In [62]:
len(ratings_meta_info.book_id.unique())

10000

In [63]:
pd.isnull(ratings_meta_info.iloc[:,7:]).all(1).sum()

0

In [66]:
ratings_meta_info.to_csv('ratings_meta_info.csv')

In [67]:
ratings_meta_info.shape

(5976479, 47)